# Regression Trees

## Objetivos

Após concluir este laboratório, você será capaz de:

Treinar uma Regression Trees

Avaliar o Desempenho de uma Árvore de Regressão

## Configuração

Para este laboratório, usaremos Python e várias bibliotecas Python. 

Algumas dessas bibliotecas podem ser instaladas em seu ambiente de laboratório ou no SN Labs. 

Outras podem precisar ser instaladas por você. 

As células abaixo instalarão essas bibliotecas quando executadas.

In [21]:
# Instalar bibliotecas que ainda não estão no ambiente usando pip
#!pip install pandas==1.3.4
#!pip install sklearn==0.20.1

ERROR: Could not find a version that satisfies the requirement sklearn==0.20.1 (from versions: 0.0, 0.0.post1, 0.0.post2, 0.0.post4, 0.0.post5, 0.0.post7, 0.0.post9, 0.0.post10, 0.0.post11, 0.0.post12)
ERROR: No matching distribution found for sklearn==0.20.1


In [22]:
# Pandas nos permitirá criar um dataframe dos dados para que eles possam ser usados e manipulados
import pandas as pd

# Algoritmo de árvore de regressão
from sklearn.tree import DecisionTreeRegressor

# Dividir nossos dados em dados de treinamento e teste
from sklearn.model_selection import train_test_split

## Sobre o conjunto de dados

Imagine que você é um cientista de dados trabalhando para uma empresa imobiliária que está planejando investir em imóveis em Boston. 

Você coletou informações sobre várias áreas de Boston e tem a tarefa de criar um modelo que pode prever o preço médio de casas para essa área para que ele possa ser usado para fazer ofertas.

O conjunto de dados tinha informações sobre áreas/cidades, não casas individuais, os recursos são

CRIM: Crime per capita

ZN: Proporção de terrenos residenciais zoneados para lotes acima de 25.000 pés quadrados.

INDUS: Proporção de acres comerciais não varejistas por cidade

CHAS: variável fictícia do Rio Charles (= 1 se o trato limita o rio; 0 caso contrário)

NOX: concentração de óxidos nítricos (partes por 10 milhões)

RM: número médio de cômodos por moradia

AGE: proporção de unidades ocupadas pelo proprietário construídas antes de 1940

DIS: distâncias ponderadas para cinco centros de emprego de Boston

RAD: índice de acessibilidade a rodovias radiais

TAX: taxa de imposto sobre a propriedade de valor integral por US$ 10.000

PTRAIO: proporção aluno-professor por cidade

LSTAT: porcentagem de status inferior da população

MEDV: valor médio de casas ocupadas pelo proprietário em US$ 1.000

In [2]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/data/real_estate_data.csv")

In [3]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,NaN,36.2


Agora vamos aprender sobre o tamanho dos nossos dados, há 506

In [4]:
data.shape

(506, 13)

A maioria dos dados é válida, há linhas com valores ausentes que lidaremos no pré-processamento

In [5]:
data.isnull().sum()

CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
LSTAT      20
MEDV        0
dtype: int64

## Pré-processamento de dados

Primeiro, vamos descartar as linhas com valores ausentes porque temos dados suficientes em nosso conjunto de dados


In [6]:
data.dropna(inplace=True)

Agora podemos ver que nosso conjunto de dados não tem valores ausentes


In [7]:
data.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
LSTAT      0
MEDV       0
dtype: int64


Vamos dividir o conjunto de dados em nossos recursos e o que estamos prevendo (alvo)

In [8]:
X = data.drop(columns=["MEDV"])
Y = data["MEDV"]

In [9]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3,222,18.7,5.21


In [10]:
Y.head()

0    24.0
1    21.6
2    34.7
3    33.4
5    28.7
Name: MEDV, dtype: float64

Finalmente, vamos dividir nossos dados em um conjunto de dados de treinamento e teste usando train_test_split de sklearn.model_selection

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=1)

## Criar Regression Trees

Árvores de Regressão são implementadas usando DecisionTreeRegressor de sklearn.tree

Os parâmetros importantes do DecisionTreeRegressor são:

- criterion: {"mse", "friedman_mse", "mae", "poisson"} - A função usada para medir o erro

- max_depth - A profundidade máxima que a árvore pode ter

- min_samples_split - O número mínimo de amostras necessárias para dividir um nó

- min_samples_leaf - O número mínimo de amostras que uma folha pode conter

- max_features: {"auto", "sqrt", "log2"} - O número de recursos que examinamos procurando o melhor, usado para acelerar o treinamento

Primeiro, vamos começar criando um objeto DecisionTreeRegressor, definindo o parâmetro criteria para mse para Erro Quadrático Médio

In [15]:
regression_tree = DecisionTreeRegressor(criterion = 'friedman_mse')

## Treinamento

Agora vamos treinar nosso modelo usando o método fit no objeto DecisionTreeRegressor fornecendo nossos dados de treinamento

In [16]:
regression_tree.fit(X_train, Y_train)

DecisionTreeRegressor(criterion='friedman_mse')

## Avaliação

Para avaliar nosso conjunto de dados, usaremos o método de pontuação do objeto DecisionTreeRegressor que fornece nossos dados de teste, este número é o valor $R^2$ que indica o coeficiente de determinação

In [17]:
regression_tree.score(X_test, Y_test)

0.8420878339438682

Também podemos encontrar o erro médio em nosso conjunto de testes, que é o erro médio na previsão do valor mediano da casa

In [18]:
prediction = regression_tree.predict(X_test)

print("$",(prediction - Y_test).abs().mean()*1000)

$ 2827.848101265822


## Exercício

Treine uma árvore de regressão usando o critério mae e então relate seu valor  $R^2$ e erro médio

In [25]:
regression_tree = DecisionTreeRegressor(criterion = "absolute_error")

In [26]:
regression_tree.fit(X_train, Y_train)


DecisionTreeRegressor(criterion='absolute_error')

In [27]:
print(regression_tree.score(X_test, Y_test))

prediction = regression_tree.predict(X_test)

print("$",(prediction - Y_test).abs().mean()*1000)

0.8799935621903793
$ 2445.5696202531644
